In [ ]:
# !pip install -qU pip wheel
# !pip install -q -r ../requirements.txt
import sys
sys.path.append('..')
from ballitoreproject import *


In [ ]:
df=get_data()
df_smpl = df # ...

In [ ]:
all_words = [word for txt in df_smpl.txt for word in txt.split()]
nltk_text = nltk.Text(all_words)
nltk_text.concordance('abolition')

In [ ]:
def concordance(word, width=100):
    all_words_with_ids = [(word,id) for id,txt in zip(df_smpl.index,df_smpl.txt) for word in txt.split()]
    all_words = [w for w,id in all_words_with_ids]
    nltk_text = nltk.Text(all_words)

    o=[]
    for match in nltk_text.concordance_list(word,lines=None, width=width):
        match_id = all_words_with_ids[match.offset][1]
        match_d={
            'id':match_id,
            'left':match.left_print,
            'word':match.query,
            'right':match.right_print
        }
        o.append(match_d)
    pd.options.display.max_colwidth=None
    return pd.DataFrame(o).set_index('id').join(df_smpl,how='left').drop('txt',axis=1)
    

In [ ]:
df_conc = concordance('abolition', width=500)
df_conc

In [ ]:
df_conc.sender.value_counts()

In [ ]:
df_conc.recipient.value_counts()

In [ ]:
# !pip install altair
import altair as alt

def plot_keyword_over_time(word, by='year', normalize=False):
    df_conc = concordance(word).query('0<year<=1950')
    df_count = df_conc[by].value_counts().reset_index()
    yearsums = df.groupby(by).num_words.sum()
    df_count['fpk'] = [count / yearsums.loc[year] * 1000 for year,count in zip(df_count[by], df_count['count'])]
    # df_count.plot(kind='scatter', x=by,y='fpk')
    
    chart = alt.Chart(df_count).mark_bar().encode(
        x=alt.X(by, title=by.capitalize()),
        y=alt.Y('count' if not normalize else 'fpk', title='Count' if not normalize else 'Count per 1000 words'),
        tooltip=[by, 'count', 'fpk']
    ).properties(
        title=f'Keyword "{word}" Count Over Time'
    )
    return chart

In [ ]:
plot_keyword_over_time('abolition')

In [ ]:
plot_keyword_over_time('doctor')

In [ ]:
plot_keyword_over_time('liberty')

In [ ]:
plot_keyword_over_time('sick')

In [ ]:
plot_keyword_over_time('slave')
